## issubclass()

### `abc.ABCMeta` + `register()`

In [1]:
from abc import ABCMeta


class MyABC1(metaclass=ABCMeta): ...


class Dummy1: ...


MyABC1.register(Dummy1)

# Dummy1 is a "virtual subclass" of MyABC1
print(issubclass(Dummy1, MyABC1))  # True
print(isinstance(Dummy1(), MyABC1))  # True

True
True


### `abc.ABC` + `register()`

In [2]:
from abc import ABC


class MyABC2(ABC): ...


class Dummy2: ...


MyABC2.register(Dummy2)

# Dummy2 is a "virtual subclass" of MyABC
print(issubclass(Dummy2, MyABC2))  # True
print(isinstance(Dummy2(), MyABC2))  # True

True
True


###  `abc.ABCMeta` + `__subclasshook__()`

In [3]:
from abc import ABCMeta


class MyABC3(metaclass=ABCMeta):
    @classmethod
    def __subclasshook__(cls, subclass):
        return True


class Dummy3: ...


# Dummy3 is a "virtual subclass" of MyABC3
print(issubclass(Dummy3, MyABC3))  # True
print(isinstance(Dummy3(), MyABC3))  # True

True
True


###  `abc.ABC` + `__subclasshook__()`

In [4]:
from abc import ABC


class MyABC4(ABC):
    @classmethod
    def __subclasshook__(cls, subclass):
        return True


class Dummy4: ...


# Dummy4 is a "virtual subclass" of MyABC4
print(issubclass(Dummy4, MyABC4))  # True
print(isinstance(Dummy4(), MyABC4))  # True

True
True


## singledispatch()

In [5]:
from functools import singledispatch


@singledispatch
def my_print(arg):
    print(f"{type(arg)=}, {arg=}")
    raise NotImplementedError(f"{type(arg)=} is not supported yet.")


@my_print.register
def _(arg: int):
    print(f"{type(arg)=}, {arg=}")


@my_print.register
def _(arg: str):
    print(f"{type(arg)=}, {arg=}")


my_print(123)  # type(arg)=<class 'int'>, arg=123
my_print("abc")  # type(arg)=<class 'str'>, arg='abc'
# my_print(1.23)  # NotImplementedError: type(arg)=<class 'float'> is not supported yet.

type(arg)=<class 'int'>, arg=123
type(arg)=<class 'str'>, arg='abc'


## Internal structure and custom fmt

In [6]:
import polars as pl
import polars.selectors as cs
from great_tables import GT
from datetime import date, time, timedelta


df_demo = pl.DataFrame(
    {
        "int": [1, 2, 3],
        "float1": [4.4, 5.5, 6.6],
        "float2": [7.7, 8.88, 9.999],
        "perc": [0.5401, -0.0895, 0.7946],
        "sci": [10001.001, -20002.002, 30003.003],
        "group": ["A", "B", "B"],
        "str": ["Python", "Ruby", "Rust"],
        "date": pl.date_range(date(2024, 9, 1), date(2024, 9, 3), eager=True),
        "time": pl.time_range(
            start=time(21, 0),
            interval=timedelta(hours=1, minutes=10),
            eager=True,
        ),
    }
).with_columns(
    pl.col("date").dt.combine(pl.col("time")).alias("datetime"),
)
df_demo

int,float1,float2,perc,sci,group,str,date,time,datetime
i64,f64,f64,f64,f64,str,str,date,time,datetime[μs]
1,4.4,7.7,0.5401,10001.001,"""A""","""Python""",2024-09-01,21:00:00,2024-09-01 21:00:00
2,5.5,8.88,-0.0895,-20002.002,"""B""","""Ruby""",2024-09-02,22:10:00,2024-09-02 22:10:00
3,6.6,9.999,0.7946,30003.003,"""B""","""Rust""",2024-09-03,23:20:00,2024-09-03 23:20:00


In [7]:
def fmt_double_plus_n_as_integer(
    self: GT,
    columns: "SelectExpr" = None,
    rows: int | list[int] | None = None,
    plus_n: str | int | float = 1,
):
    def fmt_fn(x: str | int | float | None, plus_n=plus_n):
        print(f"fmt_fn called: current {x=}")
        if x is None:
            return x
        return int(float(x) * 2 + float(plus_n))

    return self.fmt(fns=fmt_fn, columns=columns, rows=rows)


class GT2(GT):
    fmt_double_plus_n_as_integer = fmt_double_plus_n_as_integer

In [8]:
(GT2(df_demo).fmt_double_plus_n_as_integer(columns="int", plus_n=10))

fmt_fn called: current x=1
fmt_fn called: current x=2
fmt_fn called: current x=3


int,float1,float2,perc,sci,group,str,date,time,datetime
12,4.4,7.7,0.5401,10001.001,A,Python,2024-09-01,21:00:00,2024-09-01 21:00:00.000000
14,5.5,8.88,-0.0895,-20002.002,B,Ruby,2024-09-02,22:10:00,2024-09-02 22:10:00.000000
16,6.6,9.999,0.7946,30003.003,B,Rust,2024-09-03,23:20:00,2024-09-03 23:20:00.000000


In [9]:
(GT2(df_demo).fmt_double_plus_n_as_integer(columns="float1", plus_n="5"))

fmt_fn called: current x=4.4
fmt_fn called: current x=5.5
fmt_fn called: current x=6.6


int,float1,float2,perc,sci,group,str,date,time,datetime
1,13,7.7,0.5401,10001.001,A,Python,2024-09-01,21:00:00,2024-09-01 21:00:00.000000
2,16,8.88,-0.0895,-20002.002,B,Ruby,2024-09-02,22:10:00,2024-09-02 22:10:00.000000
3,18,9.999,0.7946,30003.003,B,Rust,2024-09-03,23:20:00,2024-09-03 23:20:00.000000
